In [1]:
import csv
import pandas as pd
def save_data(lips, Y, data_type):
    with open(f'{data_type}_lips.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        header = ['label']
        for i in range(lips[0].shape[0]): #frames
            for j in range(lips[0].shape[1]): #height
                for k in range(lips[0].shape[2]): #width
                    header.append(f'pixel_{i}_{j}_{k}')
        writer.writerow(header)
        for i in range(len(lips)):
            row = [Y[i].argmax()]
            current_lips = lips[i].flatten()
            for j in range(len(current_lips)):
                row.append(current_lips[j])
            writer.writerow(row)
            
            
def load_data():
    X_train = pd.read_csv('train_lips.csv')
    Y_train = X_train['label']
    del X_train['label']
    X_valid = pd.read_csv('valid_lips.csv')
    Y_valid = X_valid['label']
    del X_valid['label']
    
    X_test = pd.read_csv('test_lips.csv')
    Y_test = X_test['label']
    del X_test['label']
    return X_train, Y_train, X_valid, Y_valid, X_test, Y_test

In [2]:
# loading the data from the csv files
X_train, Y_train, X_valid, Y_valid, X_test, Y_test = load_data()
import numpy as np
import tensorflow
x_train = np.array(X_train, dtype='float32').reshape(-1, 11, 60, 100, 1)
x_valid = np.array(X_valid, dtype='float32').reshape(-1, 11, 60, 100, 1)
x_test = np.array(X_test, dtype='float32').reshape(-1, 11, 60, 100, 1)
nb_classes = 3



In [3]:
y_train = tensorflow.keras.utils.to_categorical(Y_train, nb_classes)
y_valid = tensorflow.keras.utils.to_categorical(Y_valid, nb_classes)
y_test = tensorflow.keras.utils.to_categorical(Y_test, nb_classes)


In [4]:
x_train /= 255
x_test /= 255
x_valid /=255

In [5]:
import tensorflow as tf


In [24]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv3D(64, kernel_size=(4, 5, 5), activation='relu', input_shape=(11, 60, 100, 1)))
model.add(tf.keras.layers.MaxPooling3D(pool_size=(2, 3, 4)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(nb_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [25]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d_2 (Conv3D)               │ (None, 9, 56, 96, 64)  │         4,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_2 (MaxPooling3D)  │ (None, 4, 18, 24, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 4, 18, 24, 64)  │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 110592)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │    14,155,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,169,475 (54.05 MB)

 Trainable params: 14,169,347 (54.05 MB)

 Non-trainable params: 128 (512.00 B)

In [28]:
es=tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='min', verbose=1, patience=5)

In [29]:
model.fit(x_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(x_valid, y_valid,), callbacks=[es])

Epoch 1/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 96s 2s/step - accuracy: 0.5960 - loss: 0.8618 - val_accuracy: 0.4098 - val_loss: 1.0723
Epoch 2/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - accuracy: 0.6500 - loss: 0.8057 - val_accuracy: 0.4098 - val_loss: 1.0685
Epoch 3/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - accuracy: 0.6720 - loss: 0.7482 - val_accuracy: 0.5082 - val_loss: 1.0102
Epoch 4/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 93s 2s/step - accuracy: 0.7209 - loss: 0.6546 - val_accuracy: 0.4672 - val_loss: 1.0182
Epoch 5/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 89s 2s/step - accuracy: 0.6838 - loss: 0.7554 - val_accuracy: 0.5246 - val_loss: 0.9840
Epoch 6/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 90s 2s/step - accuracy: 0.7689 - loss: 0.5705 - val_accuracy: 0.6885 - val_loss: 0.9170
Epoch 6: early stopping


In [30]:
from sklearn.metrics import classification_report

preds = model.predict(x_test)

print(classification_report(np.argmax(y_test,1),np.argmax(preds,1)))

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 272ms/step
              precision    recall  f1-score   support

           0       0.58      1.00      0.74        50
           1       1.00      0.04      0.08        23
           2       1.00      0.72      0.84        50

    accuracy                           0.71       123
   macro avg       0.86      0.59      0.55       123
weighted avg       0.83      0.71      0.65       123


In [32]:
model.save('model.simple.h5')  

In [ ]:
model= tf.keras.models.load_model('model.simple')

In [6]:

from tensorflow.keras.layers import Conv2D, Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization, Input, ReLU, GlobalAveragePooling3D, add
from tensorflow.keras.initializers import HeNormal
from keras.models import Model

output_shape = 3
input = Input(shape=(11, 60, 100, 1))

'''block_0'''
b0_conv3d_1 = Conv3D(64, kernel_size=(2, 3, 3), padding='same',
                     name='b0_conv3d_1', kernel_initializer = 'he_normal')(input)
b0_relu_1 = ReLU(name='b0_relu_1')(b0_conv3d_1)
b0_bn_1 = BatchNormalization(name='b0_bn_1')(b0_relu_1)
b0_out =  MaxPooling3D(pool_size=(2, 2, 2))(b0_bn_1)

'''block_1'''
b1_cnv3d_1 = Conv3D(filters=16, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same',
                        use_bias=False, name='b1_cnv3d_1', kernel_initializer=HeNormal())(b0_out)
b1_relu_1 = ReLU(name='b1_relu_1')(b1_cnv3d_1)
b1_bn_1 = BatchNormalization(name='b1_bn_1')(b1_relu_1)  # size: 14*14

b1_cnv3d_2 = Conv3D(filters=32, kernel_size=(1, 2, 2), strides=(2, 2, 2), padding='same',
                    use_bias=False, name='b1_cnv3d_2', kernel_initializer=HeNormal())(b1_bn_1)
b1_relu_2 = ReLU(name='b1_relu_2')(b1_cnv3d_2)
b1_out = BatchNormalization(name='b1_out')(b1_relu_2)  # size: 14*14


'''block 2'''
b2_cnv3d_1 = Conv3D(filters=32, kernel_size=(1, 2, 2), strides=(1, 1, 1), padding='same',
                    use_bias=False, name='b2_cnv3d_1', kernel_initializer=HeNormal())(b1_out)
b2_relu_1 = ReLU(name='b2_relu_1')(b2_cnv3d_1)
b2_bn_1 = BatchNormalization(name='b2_bn_1')(b2_relu_1)  

b2_add = add([b1_out, b2_bn_1])  #

b2_cnv3d_2 = Conv3D(filters=64, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same',
                    use_bias=False, name='b2_cnv3d_2', kernel_initializer=HeNormal())(b2_add)
b2_relu_2 = ReLU(name='b2_relu_2')(b2_cnv3d_2)
b2_out = BatchNormalization(name='b2_bn_2')(b2_relu_2)  # size: 7*7

'''block 3'''
b3_cnv3d_1 = Conv3D(filters=64, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='same',
                    use_bias=False, name='b3_cnv3d_1', kernel_initializer=HeNormal())(b2_out)
b3_relu_1 = ReLU(name='b3_relu_1')(b3_cnv3d_1)
b3_bn_1 = BatchNormalization(name='b3_bn_1')(b3_relu_1)  # size: 7*7

b3_add = add([b2_out, b3_bn_1])  #

b3_cnv3d_2 = Conv3D(filters=128, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same',
                    use_bias=False, name='b3_cnv3d_2', kernel_initializer=HeNormal())(b3_add)
b3_relu_2 = ReLU(name='b3_relu_2')(b3_cnv3d_2)
b3_out = BatchNormalization(name='b3_out')(b3_relu_2)  # size: 3*3


'''block 4'''
b4_avg_p = GlobalAveragePooling3D()(b3_out)
output = Dense(output_shape, name='model_output', activation='softmax',
                       kernel_initializer='he_uniform')(b4_avg_p)

model = Model(input, output)

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 11, 60,    │          0 │ -                 │
│ (InputLayer)        │ 100, 1)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b0_conv3d_1         │ (None, 11, 60,    │      1,216 │ input_layer[0][0] │
│ (Conv3D)            │ 100, 64)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b0_relu_1 (ReLU)    │ (None, 11, 60,    │          0 │ b0_conv3d_1[0][0] │
│                     │ 100, 64)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b0_bn_1             │ (None, 11, 60,    │        256 │ b0_relu_1[0][0]   │
│ (BatchNormalizatio… │ 100, 64)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d       │ (None, 5, 30, 50, │          0 │ b0_bn_1[0][0]     │
│ (MaxPooling3D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b1_cnv3d_1 (Conv3D) │ (None, 3, 15, 25, │     27,648 │ max_pooling3d[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b1_relu_1 (ReLU)    │ (None, 3, 15, 25, │          0 │ b1_cnv3d_1[0][0]  │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b1_bn_1             │ (None, 3, 15, 25, │         64 │ b1_relu_1[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b1_cnv3d_2 (Conv3D) │ (None, 2, 8, 13,  │      2,048 │ b1_bn_1[0][0]     │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b1_relu_2 (ReLU)    │ (None, 2, 8, 13,  │          0 │ b1_cnv3d_2[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b1_out              │ (None, 2, 8, 13,  │        128 │ b1_relu_2[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b2_cnv3d_1 (Conv3D) │ (None, 2, 8, 13,  │      4,096 │ b1_out[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b2_relu_1 (ReLU)    │ (None, 2, 8, 13,  │          0 │ b2_cnv3d_1[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b2_bn_1             │ (None, 2, 8, 13,  │        128 │ b2_relu_1[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 2, 8, 13,  │          0 │ b1_out[0][0],     │
│                     │ 32)               │            │ b2_bn_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b2_cnv3d_2 (Conv3D) │ (None, 1, 4, 7,   │     55,296 │ add[0][0]         │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b2_relu_2 (ReLU)    │ (None, 1, 4, 7,   │          0 │ b2_cnv3d_2[0][0]

 Total params: 317,571 (1.21 MB)

 Trainable params: 316,771 (1.21 MB)

 Non-trainable params: 800 (3.12 KB)

In [11]:
es=tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', verbose=1, patience=10, restore_best_weights=True, min_delta=0.001)

In [12]:
model.fit(x_train, y_train, batch_size=16, epochs=30, verbose=1, validation_data=(x_valid, y_valid,), callbacks=[es])

Epoch 1/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 152s 2s/step - accuracy: 0.4853 - loss: 1.1433 - val_accuracy: 0.4262 - val_loss: 1.0455
Epoch 2/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 182s 2s/step - accuracy: 0.6828 - loss: 0.6964 - val_accuracy: 0.5164 - val_loss: 1.0160
Epoch 3/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 160s 2s/step - accuracy: 0.7750 - loss: 0.5258 - val_accuracy: 0.5738 - val_loss: 0.8245
Epoch 4/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 177s 2s/step - accuracy: 0.7767 - loss: 0.5033 - val_accuracy: 0.6230 - val_loss: 0.7703
Epoch 5/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 171s 2s/step - accuracy: 0.8409 - loss: 0.3810 - val_accuracy: 0.7131 - val_loss: 0.6748
Epoch 6/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 350s 5s/step - accuracy: 0.8855 - loss: 0.2810 - val_accuracy: 0.6311 - val_loss: 1.0000
Epoch 7/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 148s 2s/step - accuracy: 0.9048 - loss: 0.2591 - val_accuracy: 0.6803 - val_loss: 0.9360
Epoch 8/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 144s 2s/step - accuracy: 0.8829 - loss: 0.2954 - val_accuracy: 0.6311 - v

In [13]:
from sklearn.metrics import classification_report

preds = model.predict(x_test)

print(classification_report(np.argmax(y_test,1),np.argmax(preds,1)))

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 299ms/step
              precision    recall  f1-score   support

           0       0.62      0.64      0.63        50
           1       0.43      0.39      0.41        23
           2       0.82      0.82      0.82        50

    accuracy                           0.67       123
   macro avg       0.62      0.62      0.62       123
weighted avg       0.66      0.67      0.66       123
